# Santa Ana Winds Indicators Notebook

Santa Ana winds are intense dry winds that blow from the Mojave Desert downslope to the Southern California coast (Seto et al. 2025). These winds are driven by a high pressure system over Mojave High Desert the rotates clockwise toward low-pressure off the California Coast. As the air descends the high desert toward the coast, it goes through dry adiabatic warming as it traverses narrow mountain canyons and passes. This process is known as the Venturi effect, where the air sinks, compresses, warms, dries out and speeds up. This creates incredibly dry air at dramatically increased wind speeds. Santa Ana wind occurrence generally peaks in December, with a season that generally spans October-March (Abel and Hall 2010). Upper level temperature and winds, along with the gradient in sea level pressure, can be used to identify conditions that support Santa Ana Wind occurrence. 

This notebook allows users to generate hourly timeseries of indicator variables for Santa Ana winds, including upper level temperature gradient, upper level winds, and sea level pressure gradient. By default two points, one in the Mojave Desert and one in Santa Monica, are used to compare these indicators. These results can then be used as inputs in the Event Finder notebook to identify timepoints where these indicators exceed a threshold.

**Intended Application:** As a user, I want to <span style="color:#FF0000">generate hourly timeseries of key Santa Ana Wind indicators: upper level temperature gradient, upper level winds, and sea level pressure gradient.
</span>
**Runtime:** On the Analytics Engine Jupyter Hub platform this notebook will take approximately **30 minutes** to run using the default settings.

**Notes:** This is a prototype notebook. Currently only 1 simulation is available for geopotential height.

**References:**

Abel, M.R., and A. Hall, 2010: Local and synoptic mechanisms causing Southern California’s Santa Ana winds. Clim Dyn 34, 847–857, https://doi.org/10.1007/s00382-009-0650-4

Seto, D., C. Jones, D. Siuta, N. Wagenbrenner, C. Thompson, and N. Quinn, 2025: Evaluation of HRRR Wind Speed Forecast and WindNinja Downscaling Accuracy during Santa Ana Wind Events in Southern California. Wea. Forecasting, 40, 525–541, https://doi.org/10.1175/WAF-D-24-0013.1. 

# Setup
### Environment

This notebook requires two packages which are not part of the standard climakitae environment. These are `h5netcdf` and `windrose`. These packages will be installed when the following cell is run:

In [ ]:
!pip install h5netcdf
!pip install windrose

Import the remaining dependencies:

In [ ]:
import climakitae as ck
from climakitae.core.constants import WRF_CRS
from climakitae.core.data_export import export
from climakitae.core.data_load import load
from climakitae.new_core.processors.warming_level import WarmingLevel
from climakitae.util.utils import add_dummy_time_to_wl, get_closest_gridcell
from climakitae.tools.derived_variables import (
    compute_wind_dir,
    compute_sea_level_pressure,
    compute_geostrophic_wind,
)

import geopandas as gpd
import numpy as np
import pyproj
from pyproj import CRS, Geod, Proj
import s3fs
import xarray as xr
from matplotlib import cm, rc_context
from matplotlib.patches import Rectangle
import matplotlib.pyplot as plt
from windrose import WindroseAxes

### Choose points
For Santa Ana wind analysis, `gradient_point_1` would typically be a location near the Southern California coast, while `gradient_point_2` would be a location in the Mojave desert.  
When a gradient variable is calculated, the value at `gradient_point_2` will be subtracted from the value at `gradient_point_1`. This is done for upper level temperature and sea level pressure. A mean of the two points will be taken for the upper level wind speed evaluation.

In [ ]:
gradient_point_1 = (34.031899244954694, -118.47507877286985)  # Santa Monica
gradient_point_2 = (34.31392420709518, -116.16003299789273)  # Mojave Desert

### Choose global warming levels
With this prototype dataset, only warming levels 0.8 and 1.5 can be used for upper level winds and temperature. Any valid warming level may be selected for sea level pressure only.

In [ ]:
baseline_gwl = 0.8
future_gwl = 1.5

# Temperature and Geostrophic Wind

## 1. Get Warming Levels

Access the WRF data for temperature and geopotential height and convert to warming levels.

In [ ]:
hist_path = "s3://santa-ana-winds/miroc6_r1i1p1f1_historical/d01/temperature_geopotential_plevs.zarr"
ssp370_path = "s3://santa-ana-winds/miroc6_r1i1p1f1_ssp370/d01/temperature_geopotential_plevs.zarr/"

ds_hist = xr.open_dataset(
    hist_path, consolidated=True, zarr_format=2, engine="zarr", chunks={}
)
ds_ssp370 = xr.open_dataset(
    ssp370_path, consolidated=True, zarr_format=2, engine="zarr", chunks={}
)

ds_hist = ds_hist.sel(time=slice("1980-09-01", "2013-12-31 23:00"))
ds_ssp370 = ds_ssp370.sel(time=slice("2014-01-01", "2066-08-04 13:00"))

ds_hist = ds_hist.expand_dims({"member_id": ["r1i1p1f1"]})
ds_ssp370 = ds_ssp370.expand_dims({"member_id": ["r1i1p1f1"]})

ds_hist = ds_hist.convert_calendar("noleap")
ds_ssp370 = ds_ssp370.convert_calendar("noleap")

# Get warming levels
values = {
    "warming_levels": [baseline_gwl, future_gwl],
    "warming_level_months": [x for x in range(1, 13)],
}
WL = WarmingLevel(values)

data = {}
data["WRF.UCLA.MIROC6.ssp370.1hr.d03"] = ds_ssp370
data["WRF.UCLA.MIROC6.historical.1hr.d03"] = ds_hist

context = {"activity_id": "WRF"}
wls = WL.execute(data, context)

ds_wl = wls["WRF.UCLA.MIROC6.ssp370.1hr.d03.r1i1p1f1"]

ds_wl.attrs["frequency"] = "1hr"
ds_wl = add_dummy_time_to_wl(ds_wl)
ds_wl = ds_wl.expand_dims("sim", axis=0)
ds_wl = ds_wl.assign_coords(
    {"sim": ["WRF.UCLA.MIROC6.ssp370.1hr.d03.r1i1p1f1".replace(".", "_")]}
)

## 3. Temperature gradient

Find the upper level temperature at the two points of interest and get the difference. Run the cell below to calculate this difference and write the results to file.

This cell may take 5+ minutes to execute depending on your compute environment.

In [ ]:
# Select a pressure level before computing the magnitude and direction of the geostrophic wind.
# Valid levels are 1000, 925, 850, 700, 500, 300, or 200. The default value is 850-200 hPa.
levels = [850, 700, 500, 300, 200]

# Get the temperature difference at selected levels
print("Getting temperature at each point.")
temperature = ds_wl["temperature"].sel(pressure_level=levels)

temp_pt_1 = get_closest_gridcell(
    temperature, gradient_point_1[0], gradient_point_1[1]
).compute()
temp_pt_2 = get_closest_gridcell(
    temperature, gradient_point_2[0], gradient_point_2[1]
).compute()

print("Calculating difference.")
temperature_diff = temp_pt_1 - temp_pt_2
temperature_diff.name = "temperature_gradient"
temperature_diff.attrs["long_name"] = "Temperature Gradient on Pressure Level"

print("Writing results to file.")
# Export hourly data, with each level as a separate file:
export(
    temperature_diff,
    f"temperature_gradient.nc",
    format="NetCDF",
    mode="local",
)

This figure shows the temperature gradient at the selected level for each warming level at the selected pressure level (default is 850 hPa).

In [ ]:
# Pick a pressure level to display
level = 850  # hPa

f, ax = plt.subplots(1, 2, figsize=(10, 4))
plt.rcParams.update(
    {
        "grid.color": "0.5",  # gray gridlines
        "grid.linestyle": "--",  # solid gridlines
        "grid.linewidth": 0.5,  # thin gridlines
    }
)


temperature_diff.sel(warming_level=baseline_gwl).plot(ax=ax[0])
ax[0].set_title(f"GWL {baseline_gwl}")
ax[0].set_ylim([-15, 20])
ax[0].set_xlim([temperature_diff.time[0].data, temperature_diff.time[-1].data])
ax[0].grid(True)

temperature_diff.sel(warming_level=future_gwl).plot(ax=ax[1])
ax[1].set_title(f"GWL {future_gwl}C")
ax[1].set_ylim([-15, 20])
ax[1].set_xlim([temperature_diff.time[0].data, temperature_diff.time[-1].data])
ax[1].grid(True)

plt.suptitle(f"Gradient of {level}hPa temperature")
plt.tight_layout()
plt.savefig(f"temperature_gradient_{level}hPa_by_gwl.png")

## 4. Wind on Pressure Levels
In this section we use the geostrophic wind equation to calculate the upper level wind on pressure levels at a point in between the two points of interest.

In the cell below, assign a pressure level in hectopascal (hPa) units to the `level` variable.

In [ ]:
# Select a pressure level before computing the magnitude and direction of the geostrophic wind.
# Valid levels are 1000, 925, 850, 700, 500, 300, or 200. The default value is 850-200 hPa.
levels = [850, 700, 500, 300, 200]

Now run the following cell to save geostrophic wind magnitude and direction timeseries files for this level. This cell may take 10+ minutes to execute depending on your compute environment. 

In [ ]:
# The geostrophic wind will be calculated for each point for the selected level.
for point_ind, point in enumerate([gradient_point_1, gradient_point_2]):
    print(f"Working on point {point_ind+1}")

    # Pull out the selected pressure level
    print(f"Calculating geostrophic wind for levels {levels}")
    geopotential_height = ds_wl["geopotential_height"].sel(pressure_level=levels)

    # Get the geostrophic wind components at this point
    geo_wind_u_earth, geo_wind_v_earth = compute_geostrophic_wind(geopotential_height)

    print("Calculating magnitude and direction.")
    # Extract timeseries for just the point of interest
    geo_u_level = load(
        get_closest_gridcell(geo_wind_u_earth, point[0], point[1]), progress_bar=True
    )
    geo_v_level = load(
        get_closest_gridcell(geo_wind_v_earth, point[0], point[1]), progress_bar=True
    )

    # Calculate magnitude
    magnitude = np.sqrt(geo_u_level**2 + geo_v_level**2)
    magnitude.name = "magnitude"
    magnitude.attrs["long_name"] = "Geostrophic Wind Magnitude"
    magnitude.attrs["units"] = "m/s"

    # Calculate direction
    direction = compute_wind_dir(geo_u_level, geo_v_level)
    direction.name = "direction"
    direction.attrs["long_name"] = "Geostrophic Wind Direction"
    # Don't need to add units manually; units were added by compute_wind_dir

    print("Saving results to file.")
    # Export the timeseries for this level to file. Using the default settings, each of these files is
    # approximately 4 Mb in size (30 years of hourly data for two warming levels).
    export(
        magnitude,
        f"geostrophic_wind_magnitude_point_{point_ind+1}.nc",
        format="NetCDF",
        mode="local",
    )
    export(
        direction,
        f"geostrophic_wind_direction_point_{point_ind+1}.nc",
        format="NetCDF",
        mode="local",
    )
    del geo_u_level, geo_v_level, magnitude, direction
    print("\n")

# Finally, reload the magnitude timeseries and find the average of the
# wind speed at the two points.
print(f"Calculating 2-point mean wind magnitude for levels {levels}.")
magnitude_1 = xr.open_dataset(f"geostrophic_wind_magnitude_point_1.nc")
magnitude_2 = xr.open_dataset(f"geostrophic_wind_magnitude_point_2.nc")
magnitude_mean = (magnitude_1 + magnitude_2) / 2
export(
    magnitude_mean,
    f"geostrophic_wind_magnitude_point_mean.nc",
    format="NetCDF",
    mode="local",
)

print("\nDone with geostrophic wind.")

Optionally, this code can be used to visualize the upper level wind results.

In [ ]:
plot_level = 850

f, ax = plt.subplots(1, 2, figsize=(10, 4))
plt.rcParams.update(
    {
        "grid.color": "0.5",  # gray gridlines
        "grid.linestyle": "--",  # solid gridlines
        "grid.linewidth": 0.5,  # thin gridlines
    }
)

magnitude = xr.open_dataset(f"geostrophic_wind_magnitude_{plot_level}hPa_point_mean.nc")
magnitude["magnitude"].sel(warming_level=baseline_gwl).plot(ax=ax[0])
ax[0].set_title(f"GWL {baseline_gwl}")
ax[0].set_ylim([0, 70])
ax[0].set_xlim([magnitude.time[0].data, magnitude.time[-1].data])
ax[0].grid(True)

magnitude["magnitude"].sel(warming_level=future_gwl).plot(ax=ax[1])
ax[1].set_title(f"GWL {future_gwl}C")
ax[1].set_ylim([0, 70])
ax[1].set_xlim([magnitude.time[0].data, magnitude.time[-1].data])
ax[1].grid(True)

plt.suptitle(f"Point mean {plot_level}hPa wind magnitude")
plt.tight_layout()
plt.savefig(f"wind_mean_magnitude_{plot_level}hPa_by_gwl.png")

The following cell will generate wind roses for point 1 and point 2.

The wind direction at `gradient_point_1` (coast) can indicate if the wind is blowing offshore or not. In this case, the wind vector direction indicates the direction from which the wind blows, following meteorological convention (0-180° being an easterly wind).

In [ ]:
plot_level = 850

for point_ind in [1, 2]:
    wd_data = xr.open_dataset(
        f"geostrophic_wind_direction_{plot_level}hPa_point_{point_ind}.nc"
    )["direction"]
    ws_data = xr.open_dataset(
        f"geostrophic_wind_magnitude_{plot_level}hPa_point_{point_ind}.nc"
    )["magnitude"]

    title = f"{plot_level}hPa Wind Rose at Point {point_ind}"
    rmax = 24  # Max percent displayed
    rs = 6  # Percent spacing
    bins = np.arange(0, 18, 3)  # Wind speed bins

    rcParams = {
        "figure.figsize": (6, 6),
        "figure.facecolor": "white",
        "axes.facecolor": "lightgrey",
        "axes.edgecolor": "white",
        "grid.color": "white",
    }

    with rc_context(rcParams):
        fig, ax = plt.subplots(subplot_kw={"projection": "windrose"})

        ax.bar(
            wd_data.values.ravel(),
            ws_data.values.ravel(),
            bins=bins,
            cmap=cm.plasma,
            normed=True,
            opening=0.8,
            edgecolor="none",
            alpha=1,
            calm_limit=None,
        )

        # Fix patch layering
        minzorder = min(
            [c.get_zorder() for c in ax.get_children() if isinstance(c, Rectangle)]
        )

        for child in ax.get_children():
            if isinstance(child, Rectangle):
                zorder = child.get_zorder()
                child.set_zorder(3 + (zorder - minzorder))

        # Set labels
        labels = [
            "E",
            "ENE",
            "NE",
            "NNE",
            "N",
            "NNW",
            "NW",
            "WNW",
            "W",
            "WSW",
            "SW",
            "SSW",
            "S",
            "SSE",
            "SE",
            "ESE",
        ]

        # Set theta grid locations every 11.25 degrees
        angles = np.arange(0, 360, 45 / 2)
        ax.set_thetagrids(angles, labels, fontsize=10)

        ax.set_title(title)
        ax.set_legend(loc=4)

        # Set consistent magnitude scale
        ax.set_rmax(rmax)
        ticks = np.arange(rs, rmax + rs, rs, dtype="int")
        ax.set_yticks(ticks)
        ax.set_yticklabels([f"{p}%" for p in ticks])

        plt.show()

        plt.tight_layout()
        fig.savefig(f"wind_rose_point_{point_ind}_{plot_level}hPa_by_gwl.png")

# Sea Level Pressure

Sea level is computed using surface pressure, air temperature, and mixing ratio. Then the difference between two points is calculated and saved to file.

### Setup
Choose the time frequency in the cell below. This section will use the warming levels and locations selected in the "Setup" section at the top of the notebook.

In [ ]:
frequency = "1hr"  # 1hr or day

This function will handle the sea level pressure calculation for the selection gridpoints.

In [ ]:
def get_slp_at_point(
    psfc: xr.DataArray,
    t2: xr.DataArray,
    q2: xr.DataArray,
    elevation: xr.DataArray,
    point: tuple[float],
) -> xr.DataArray:
    """Extract point of interest and return sea level pressure at that point.

    This function uses the default settings for `compute_sea_level_pressure`
    including for lapse rate.

    Parameters
    ----------
        surface_pressure : xr.DataArray
            Surface pressure in Pascals
        air_temperature : xr.DataArray
            Surface air temperature in Kelvin
        mixing_ratio : xr.DataArray
            Surface mixing ratio
        elevation : xr.DataArray
            Elevation in meters

    Returns
    -------
    xr.DataArray
        Sea level pressure in Hectopascals
    """

    # Extract data at point of interest
    print("Selecting point from input arrays.")
    psfc_pt = get_closest_gridcell(psfc.rio.write_crs(WRF_CRS), point[0], point[1])
    t2_pt = get_closest_gridcell(t2.rio.write_crs(WRF_CRS), point[0], point[1])
    q2_pt = get_closest_gridcell(q2.rio.write_crs(WRF_CRS), point[0], point[1])
    elev_pt = get_closest_gridcell(elevation.rio.write_crs(WRF_CRS), point[0], point[1])

    print("Calculating sea level pressure.")
    if frequency == "1hr":
        slp_pt = compute_sea_level_pressure(psfc_pt, t2_pt, q2_pt, elev_pt)
        slp_pt.attrs["frequency"] = "hourly"
    else:
        slp_pt = compute_sea_level_pressure(
            psfc_pt, t2_pt, q2_pt, elev_pt, average_t2=False
        )
        slp_pt.attrs["frequency"] = "daily"

    # Convert to hPa
    slp_pt = slp_pt / 100.0
    slp_pt.attrs["units"] = "hPa"

    # Add dummy time axis
    slp_pt = add_dummy_time_to_wl(slp_pt)

    return slp_pt

Access the three variables (surface pressure, air temperature, and mixing ratio) we need to get sea level pressure. Results are clipped to the two points of interest. This cell will take several minutes to run.

In [ ]:
cd = ck.ClimateData(verbosity=-1)

# Surface pressure
psfc = (
    cd.catalog("cadcat")
    .activity_id("WRF")
    .institution_id("UCLA")
    .table_id(frequency)
    .grid_label("d03")
    .variable("psfc")
    .processes(
        {
            "clip": [gradient_point_1, gradient_point_2],
            "warming_level": {
                "warming_levels": [baseline_gwl, future_gwl],
            },
        }
    )
    .get()
)

# Air temperature
t2 = (
    cd.catalog("cadcat")
    .activity_id("WRF")
    .institution_id("UCLA")
    .table_id(frequency)
    .grid_label("d03")
    .variable("t2")
    .processes(
        {
            "clip": [gradient_point_1, gradient_point_2],
            "warming_level": {
                "warming_levels": [baseline_gwl, future_gwl],
            },
        }
    )
    .get()
)

# Mixing ratio
q2 = (
    cd.catalog("cadcat")
    .activity_id("WRF")
    .institution_id("UCLA")
    .table_id(frequency)
    .grid_label("d03")
    .variable("q2")
    .processes(
        {
            "clip": [gradient_point_1, gradient_point_2],
            "warming_level": {
                "warming_levels": [baseline_gwl, future_gwl],
            },
        }
    )
    .get()
)

Get the elevation data for the d03 grid.

In [ ]:
fs = s3fs.S3FileSystem(anon=True)
elevation_url = "s3://cadcat/wrf/cae/elevation_wrf.nc"
elevation = xr.open_dataset(fs.open(elevation_url), engine="h5netcdf")

Calculate the sea level pressure at each point and get the difference.

In [ ]:
# Get SLP at points
slp_pt_1 = get_slp_at_point(
    psfc["psfc"], t2["t2"], q2["q2"], elevation["elevation"], gradient_point_1
).compute()
slp_pt_2 = get_slp_at_point(
    psfc["psfc"], t2["t2"], q2["q2"], elevation["elevation"], gradient_point_2
).compute()

# Get difference
slp_dif = slp_pt_1 - slp_pt_2

# Convert to dataset with name "slp_gradient"
slp_ds = slp_dif.to_dataset(name="slp_gradient")

export(slp_ds, "slp_gradient.nc", format="NetCDF", mode="local")

This figure demonstrates the resulting gradient for a single model for one year from the baseline warming level.

In [ ]:
f, ax = plt.subplots(1, 2, figsize=(10, 4))
plt.rcParams.update(
    {
        "grid.color": "0.5",  # gray gridlines
        "grid.linestyle": "--",  # solid gridlines
        "grid.linewidth": 0.5,  # thin gridlines
    }
)


slp_ds["slp_gradient"].sel(
    warming_level=baseline_gwl, sim="WRF_UCLA_MIROC6_ssp370_1hr_d03_r1i1p1f1"
).plot(ax=ax[0])
ax[0].set_title(f"GWL {baseline_gwl}")
# ax[0].set_ylim([0, 70])
ax[0].set_xlim([slp_ds.time[0].data, slp_ds.time[-1].data])
ax[0].grid(True)

slp_ds["slp_gradient"].sel(
    warming_level=future_gwl, sim="WRF_UCLA_MIROC6_ssp370_1hr_d03_r1i1p1f1"
).plot(ax=ax[1])
ax[1].set_title(f"GWL {future_gwl}C")
# ax[1].set_ylim([0, 70])
ax[1].set_xlim([slp_ds.time[0].data, slp_ds.time[-1].data])
ax[1].grid(True)

plt.suptitle(f"Sea level pressure gradient\nWRF_UCLA_MIROC6_ssp370_1hr_d03_r1i1p1f1")
plt.tight_layout()
plt.savefig(f"slp_gradient_by_gwl_WRF_UCLA_MIROC6_ssp370_1hr_d03_r1i1p1f1.png")